In [1]:
import numpy as np
import QuantLib as ql
from scipy.integrate import quad

In [2]:
# First let's match a vanilla price, which simplifies things
K, spot = 100, 100
T, t = 1.0, 0.0
r, q = 0.0, 0.0

t_n = [T]
n = len(t_n)

vol = 0.3
v0 = vol * vol
kappa = 1.15
theta = 0.0348
rho = -0.64
sigma = 0.39

# global lookup
global lookup

lookup = {}

In [3]:
# Intermediate params

k_star = 0

for i, t_star in enumerate(t_n):
    if t > t_star:
        k_star = i+1

def tau_k(t, k):
    if k < k_star or k > n+1:
        raise KeyError("tau_k: k is out of range.. {}".format(k))
    if k == k_star:
        return t
    if k == n+1:
        return T

    return t_n[k-1]

def omega(s, w, k):
    if k < k_star or k > n+1:
        raise KeyError("omega_k: k is out of range.. {}".format(k))
    if k == k_star:
        return 0
    if k == n+1:
        return rho*w/sigma

    return rho*s/(sigma*n)

k_star

0

In [4]:
def z(s,w,k):
    term_1 = 0.5 * (2*rho*kappa - sigma)*((n-k+1)*s + w*n) / (sigma*n)
    term_2 = 0.5 * (1 - rho**2) * ((n-k+1)*s + n*w)**2 / n**2

    return term_1 + term_2

def a(s,w,t):
    term_1 = (s*(n-k_star)/n + w) * (np.log(spot) - rho*v0/sigma - t*(r - rho*kappa*theta/sigma))
    term_2 = (r - rho*kappa*theta/sigma) * (s*(np.sum(t_n))/n + w*T)

    return term_1 + term_2

def F(z1, z2, tau):
    x = np.sqrt(kappa**2 - 2*z1*sigma**2)

    term_1 = np.cosh(0.5 * tau * x)
    term_2 = (kappa - z2 * sigma**2) * np.sinh(0.5 * tau * x) / x

    return term_1 + term_2

def F_tilde(z1, z2, tau):
    x = np.sqrt(kappa**2 - 2*z1*sigma**2)
    term_1 = 0.5 * x * np.sinh(0.5 * tau * x)
    term_2 = 0.5 * (kappa - z2 * sigma**2) * np.cosh(0.5 * tau * x)

    return term_1 + term_2

In [5]:
def omega_tilde(s, w, k):
    if k < k_star or k > n+1:
        raise KeyError("omega_tilde: k is out of range.. {}".format(k))

    omega_k = omega(s, w, k)

    if k == n+1:
        return omega_k

    tau_temp = tau_k(t, k+1) - tau_k(t, k)
    z_k = z(s, w, k+1)

    if (s, w, k+1) in lookup:
        omega_tilde_k = lookup[(s, w, k+1)]
    else:
        omega_tilde_k = omega_tilde(s, w, k+1)

    result = omega_k + kappa/sigma**2 - 2*F_tilde(z_k,omega_tilde_k,tau_temp)/(F(z_k,omega_tilde_k,tau_temp)*sigma**2)
    lookup[(s, w, k)] = result
    return result

In [6]:
def Phi(s, w, t):
    term_1 = a(s,w,t) + omega_tilde(s,w,k_star) * v0
    term_2 = kappa**2 * theta * (T-t) / sigma**2
    term_3 = 2 * kappa * theta / sigma**2

    summation = 0
    for k in range(k_star+1, n+2):
        tau_temp = tau_k(t, k) - tau_k(t, k-1)
        summation += np.log(F(z(s,w,k), omega_tilde(s,w,k), tau_temp))

    return np.exp(term_1 + term_2 - term_3 * summation)

def integrand(epsilon):
    term_1 = Phi(1 + epsilon*1j,0,0) - K * Phi(epsilon*1j,0,0)
    term_2 = np.exp(-1j*epsilon*np.log(K))/(epsilon*1j)

    return np.real(term_1 * term_2)

In [7]:
# Calculated Prices
forward = Phi(1, 0, 0)
call_price = np.exp(-r)*((Phi(1, 0, 0) - K) / 2 + quad(integrand,  1e-8, 120, limit=10000)[0] / np.pi)
put_price = np.exp(-r)*((K - Phi(1, 0, 0)) / 2 + quad(integrand, 1e-8, 120, limit=10000)[0] / np.pi)

forward, call_price, put_price

(99.99999999999996, 9.609183144640326, 9.609183144640369)

In [8]:
#Explicit Vanilla Prices - MATCH!!!
print("v0: {}, kappa: {}, theta: {}, rho: {}, sigma: {}, r: {}, spot: {}, K: {}".format(v0, kappa, theta, rho, sigma, r, spot, K))

today = ql.Date(1, 12, 2020)
day_count = ql.Actual365Fixed()
expiry_date = today + ql.Period(12, ql.Months)

rTS = ql.YieldTermStructureHandle(ql.FlatForward(today, r, day_count))
qTS = ql.YieldTermStructureHandle(ql.FlatForward(today, q, day_count))
spot_quote = ql.QuoteHandle(ql.SimpleQuote(spot))

heston_process = ql.HestonProcess(rTS, qTS, spot_quote, v0, kappa, theta, sigma, rho)
heston_model = ql.HestonModel(heston_process)
heston_engine = ql.AnalyticHestonEngine(heston_model)

european_exercise = ql.EuropeanExercise(expiry_date)
vanilla_payoff = ql.PlainVanillaPayoff(ql.Option.Call, K)
vanilla_payoff_put = ql.PlainVanillaPayoff(ql.Option.Put, K)

vanilla = ql.VanillaOption(vanilla_payoff, european_exercise)
vanilla_2 = ql.VanillaOption(vanilla_payoff_put, european_exercise)

vanilla.setPricingEngine(heston_engine)
vanilla_2.setPricingEngine(heston_engine)

vanilla.NPV(), vanilla_2.NPV()

v0: 0.09, kappa: 1.15, theta: 0.0348, rho: -0.64, sigma: 0.39, r: 0.0, spot: 100, K: 100


(9.609183165210446, 9.609183165210446)

In [9]:
# Now match to a result in the paper - 1.0y, K=100 - Price: 7.2243
K, spot = 100, 100
T, t = 1.0, 0.0
r, q = 0.05, 0.0

t_n = np.linspace(0, T, 52)[1:]
n = len(t_n)

vol = 0.3
v0 = vol * vol
kappa = 1.15
theta = 0.0348
rho = -0.64
sigma = 0.39

lookup = {}

In [10]:
# Intermediate params, calculate properly

k_star = 0

for i, t_star in enumerate(t_n):
    if t > t_star:
        k_star = i+1

def tau_k(t, k):
    if k < k_star or k > n+1:
        raise KeyError("tau_k: k is out of range.. {}".format(k))
    if k == k_star:
        return t
    if k == n+1:
        return T

    return t_n[k-1]

def omega(s, w, k):
    if k < k_star or k > n+1:
        raise KeyError("omega_k: k is out of range.. {}".format(k))
    if k == k_star:
        return 0
    if k == n+1:
        return rho*w/sigma

    return rho*s/(sigma*n)

k_star, tau_k(0.0, 0), tau_k(0.0, 1), tau_k(0.0, 2)

(0, 0.0, 0.0196078431372549, 0.0392156862745098)

In [11]:
# Calculated Prices
asian_forward_price = Phi(1, 0, 0)
call_price = np.exp(-r)*((Phi(1, 0, 0) - K) / 2 + quad(integrand,  1e-8, 120, limit=10000)[0] / np.pi)
put_price = np.exp(-r)*((K - Phi(1, 0, 0)) / 2 + quad(integrand, 1e-8, 120, limit=10000)[0] / np.pi)

asian_forward_price, call_price, put_price

(102.0544608281353, 7.226438177679556, 5.27217458647315)